In [ ]:
import pandas as pd
import numpy as np
import os
import xarray as xr
import matplotlib.pyplot as plt
import glob

### Process EOBS data to mHM

In [3]:
# Define variable mappings
vars = ["rr", "tx", "tn", "tg"]
mhm_varsnames = ["pre", "tmax", "tmin", "tavg"]

src_dir = "D:/VUB/_data/EOBS_metdata"
output_dir = os.path.join(src_dir, "mhm_format")
os.makedirs(output_dir, exist_ok=True)  # Ensure output directory exists

for var, var_name in zip(vars, mhm_varsnames):
    var_files = glob.glob(os.path.join(src_dir, f"*{var}_*.nc"))
    
    if not var_files:
        print(f"Warning: No file found for {var}")
        continue  # Skip if no matching file
    
    var_path = var_files[0]
    ds = xr.open_dataset(var_path)

    # Verify coordinate names
    lon_name = "longitude" if "longitude" in ds.coords else "lon"
    lat_name = "latitude" if "latitude" in ds.coords else "lat"

    # Clip to mHM domain
    lon_min, lon_max, lat_min, lat_max = 2.0, 7.0, 48.5, 52.0
    ds_domain = ds.sel({lon_name: slice(lon_min, lon_max), lat_name: slice(lat_min, lat_max)})

    # Rename variables to mHM convention
    ds_domain = ds_domain.rename({var: var_name})

    # Select data from 1950 to 2023
    ds_domain = ds_domain.sel(time=slice("1950-01-01", "2023-12-31"))

    # # # Save to NetCDF
    # outfile = os.path.join(output_dir, f"{var_name}_raw.nc")
    # ds_domain.to_netcdf(outfile, mode="w")

In [9]:
lat = ds_domain.latitude

lat = np.tile(lat, (40, 1))  # Expand to (40, 28)
lat = lat[None, :, :]  # Reshape to (1, 40, 28) for broadcasting

In [14]:
#calculate pet using Hargreaves Samani method

def hargreaves_samani(tmin, tmax, lat, tavg=None):
    """
    Calculate potential evapotranspiration (PET) using the Hargreaves-Samani method.

    Parameters:
    tmin (xarray.DataArray): Minimum temperature (°C).
    tmax (xarray.DataArray): Maximum temperature (°C).
    lat (xarray.DataArray): Latitude (degrees).
    tavg (xarray.DataArray, optional): Average temperature (°C).

    Returns:
    xarray.DataArray: Potential evapotranspiration (mm/day).
    """
    # Convert latitude to radians
    lat_rad = np.radians(lat)

    # Calculate the mean temperature if not provided
    if tavg is None:
        tavg = (tmin + tmax) / 2.0

    # Calculate the day of the year
    doy = ds_domain.time.dt.dayofyear

    # Calculate the extraterrestrial radiation
    dr = 1 + 0.033 * np.cos(2 * np.pi * doy / 365.0)
    delta = 0.409 * np.sin((2 * np.pi / 365) * doy - 1.39)
    ra = (24 * 60 / np.pi) * 0.0820 * dr * (
        np.sin(lat_rad) * np.sin(delta) + np.cos(lat_rad) * np.cos(delta) * np.sin(2 * np.pi * doy / 365.0)
    )

    # Calculate PET using Hargreaves-Samani method
    pet = 0.0023 * ra * ((tmax - tmin) ** 0.5) * ((tavg + 17.8))

    return pet

In [15]:
doy = ds_domain.time.dt.dayofyear